I am creating a single trace in a transductive setting, where the train part of the trace dataset contains all the entities and relations

In [1]:
import pandas as pd
import numpy as np
import os
import joblib
import zipfile
from tqdm import tqdm
import contextlib
import asposecells
import jpype
from asposecells.api import Workbook, FileFormatType

seed = 1234

In [2]:

# # specify the name of the zip file and the directory to extract the contents
# zip_file_name = f'MSCallGraph_joined_reduced.zip'
# extract_dir = ""

# # create a ZipFile object and extract the contents to the specified directory
# with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)


In [3]:
complete_df = pd.read_csv('MSCallGraph_joined_reduced.csv').drop(['Unnamed: 0'],axis=1)
complete_df

,traceid,um,rpctype,dm
0,0b133c1915919238193454000e5d37,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...
1,0b133c1915919238193454000e5d37,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc,fd6d86bd0fd550e717c1fdb82a33190a9fef216d87d535...
2,0b133c1915919238193454000e5d37,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,userDefined,5cca70246befb1f4c9546d2912b9419dee54439218efa5...
3,0b133c1915919238193454000e5d37,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,84f9f68ef003a21288fffe8f9a09a5a29b05f4cc4229b8...
4,0b133c1915919238193454000e5d37,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc,01d660afcfadafd587e20ec4c04ddbc7eb0de95643ba0e...
...,...,...,...,...
1637083,0b52069415919238409694000ea18b,9a9e8613b6d7d1b573ba38dba19eec9c3cc460ab00cb49...,rpc,0d4e0aa3996cbc207424a87d8ce16610b7694e2fac37d7...
1637084,015101cf15919238987695000e,35114acfb54c54fb9618f23cd28bbc57c765f597df1409...,mc,9653f5baba69c9fb50bfb30a8571eb04dbceaae7c7f379...
1637085,0b52069415919238409694000ea18b,614c66b178d3cfe299b11254ecb4321f85beca4b2ccac8...,mc,4bba5c2371c3384862e69615979c2aa5b7f1b4fc3ea914...
1637086,0b52069415919238409694000ea18b,614c66b178d3cfe299b11254ecb4321f85beca4b2ccac8...,rpc,9a9e8613b6d7d1b573ba38dba19eec9c3cc460ab00cb49...


In [4]:
# triplets_df = complete_df.drop(['traceid'],axis=1)
# triplets_df

In [5]:
# df = triplets_df
df = complete_df.drop(['traceid'],axis=1)
df = df.drop_duplicates()
df

,um,rpctype,dm
0,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...
1,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc,fd6d86bd0fd550e717c1fdb82a33190a9fef216d87d535...
2,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,userDefined,5cca70246befb1f4c9546d2912b9419dee54439218efa5...
3,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,84f9f68ef003a21288fffe8f9a09a5a29b05f4cc4229b8...
4,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc,01d660afcfadafd587e20ec4c04ddbc7eb0de95643ba0e...
...,...,...,...
1635420,0674d4c8b48f44a836019adff9dffe0cce2ee6e7b93ee0...,db,039ff41dc3636105ffd26b0ec675eec246a965e8102c77...
1635422,1cf86b19c487cc10ea3bb9623a319454029e040d0f4e9d...,db,bb61ac88608568ac2017c8559c1da973f4f94c7d1a958e...
1635427,2c52c868f36f3920d201fddda80c8062998ebc07ee2b85...,db,abb42d7303a4376befc681f9b4a765c6b6575de0df98ed...
1635434,1cf86b19c487cc10ea3bb9623a319454029e040d0f4e9d...,db,b0748973546ca26a9cbfe7802e52924291a76929a704a4...


In [6]:
entities = set(df['dm']) | set(df['um'])
len(entities)

7386

In [7]:
baseline = []
for entity in entities:
    baseline.append(df[df['dm'] == entity].head(1))
    baseline.append(df[df['um'] == entity].head(1))

In [8]:
baseline_df = pd.concat(baseline)
baseline_df = baseline_df.drop_duplicates()
baseline_df

,um,rpctype,dm
42117,48e8faba4a168f803d7f16abb37ff877edd589b8081166...,db,dae9ae0bf7dc3f99719357729b4723707a158061468616...
206942,6ca85b42f7a72a0442352088bcc16a12bd57613edd565c...,userDefined,6ca85b42f7a72a0442352088bcc16a12bd57613edd565c...
162382,6ca85b42f7a72a0442352088bcc16a12bd57613edd565c...,mq,2d70fe8a64f53c8548b63a0435df270ae222f4e370fcbc...
258238,6f83eed1237572824fa0807fb2414de831c989c6392cca...,mc,c29ccad35343cb8d292748e45c3def0ce9ac5bcd05734a...
310485,4044a5c14fb439a9de91ea9e75f80d83fcc6c31590f8b4...,db,4e302ee7a7b2d902277a5658a38d465ae7c9d595831353...
...,...,...,...
153488,7a1c153f334a63ae491e7fbf303162576d0894933eab3d...,db,493392c0a233c1b9125f1601d4ebec14667e0c2a186b77...
392877,01b68874430d73c15b5f85ef48e66adea74c2011f413fb...,db,b07211f03eb8a63b545097cae49525e3dac2f6b1065c7e...
304964,222fb26bce12a45b062ee2476eea504a3a88ab800f1704...,db,56f604f5b995c9e6f90807ec6d0b2d59a78b25eae76581...
311487,f9014fe2e1ad08400d71587fd3d9d8d3e19846d8b7e74e...,rpc,5850621f543d3b2fa49c4e40cd71d2737865c5a098dd4a...


In [9]:
n_train_items = int(len(df)*0.64)
n_items_to_add = n_train_items - len(baseline_df)
print("the number of items to add to the train set is: ",n_items_to_add)

the number of items to add to the train set is:  2185


In [10]:
not_baseline_df = df.merge(baseline_df, how='outer', indicator=True).loc[lambda x : x['_merge']=='left_only'].drop('_merge',axis= 1)
not_baseline_df

,um,rpctype,dm
9,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,userDefined,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...
11,4ab265f54516248ee8873be7d6441912456ce17e84f399...,rpc,091794afdcf1abbaf62404337ff72a38df96c5d8af8bd0...
12,4ab265f54516248ee8873be7d6441912456ce17e84f399...,http,091794afdcf1abbaf62404337ff72a38df96c5d8af8bd0...
19,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,e6c95bef37d936ebb375bf135ded88c96eb9257a1d3d9d...
20,4ab265f54516248ee8873be7d6441912456ce17e84f399...,rpc,5cca70246befb1f4c9546d2912b9419dee54439218efa5...
...,...,...,...
16208,f2ff782d57371c090c294859e444ee4bdfb4b317e68368...,db,a2a5a9a0492f495f3dcc0bd933b0819cc8ac14289059d0...
16209,5019702f16bab0ec5627583db3cf7def36cbd87674088b...,db,6115254f36ed41a883d751b18e2bf78cc1c1ec76cd3da2...
16211,5b563df180af0e674902ffbeddf022580eee216f0a172f...,db,1abd717ffef1e3721f131eb3b37fc5c84fb9c52d0a42f3...
16213,9a0f0dd3e278f678cfa3f874a7c74af8206de653ec0396...,rpc,5ef671a7799d52d75c5c1b87d3542b5619c0ca7c9b3d80...


In [11]:
not_baseline_train = not_baseline_df.sample(n_items_to_add,random_state = seed)
train_df = pd.concat([baseline_df, not_baseline_train])
train_df

,um,rpctype,dm
42117,48e8faba4a168f803d7f16abb37ff877edd589b8081166...,db,dae9ae0bf7dc3f99719357729b4723707a158061468616...
206942,6ca85b42f7a72a0442352088bcc16a12bd57613edd565c...,userDefined,6ca85b42f7a72a0442352088bcc16a12bd57613edd565c...
162382,6ca85b42f7a72a0442352088bcc16a12bd57613edd565c...,mq,2d70fe8a64f53c8548b63a0435df270ae222f4e370fcbc...
258238,6f83eed1237572824fa0807fb2414de831c989c6392cca...,mc,c29ccad35343cb8d292748e45c3def0ce9ac5bcd05734a...
310485,4044a5c14fb439a9de91ea9e75f80d83fcc6c31590f8b4...,db,4e302ee7a7b2d902277a5658a38d465ae7c9d595831353...
...,...,...,...
5560,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,4ed7c6414695b486064dc4611c135492c4459ce9f3edbd...
12393,f10612cc80a4777bcb0f9025041b3e4a9d8de8b20bfd2d...,db,c4295080f8c580f6c929788ab68d53622b899d1d3de90d...
3164,aa42d395e6b2ab2146c4f60800b1463b0b4eec59147ff5...,mc,8f8da7913c052992423534e5b9f49e72b4213ea8fd10db...
10578,377f1c6e2b6c3315132d0d8dbd721dc01630e5919f39b0...,rpc,f10612cc80a4777bcb0f9025041b3e4a9d8de8b20bfd2d...


In [12]:
remaining_df = not_baseline_df.drop(not_baseline_train.index) 

In [13]:
validation_df = remaining_df.sample(frac = 0.44, random_state = seed)
validation_df

,um,rpctype,dm
6921,99f2e7b501f50db9b4089242a5d3e1aba334c32e0c718b...,mq,2a8dcc1d92fecd5d42d5be4a5d6c443a97b377526ad778...
12980,5b563df180af0e674902ffbeddf022580eee216f0a172f...,db,635a68af6f78ae2c3ca0832f23f3f5dfbfb54200b00e7e...
16191,87f3522e7b259f45398e495f02e0afd54d9bd8f77959c6...,db,ce22bc7ea1cf2f5c71c3202976d7a4e3f10d74494efa40...
209,4d980b8ae1f1da1a302d77ef91b9f9dd4e841856f6cdd4...,mc,0b0c981da8042a436f30f7d9f7cce74360abe630bd5478...
10200,2888a351e30955adb09e56e906c415036ab8b10f9ebc16...,http,2bbca6dc6ba99115e631eba25efc62743aff6d9fe3fe3a...
...,...,...,...
13051,e2350413f1606f6332ee2a2d98f09029cd454b0b216b24...,rpc,f2ff782d57371c090c294859e444ee4bdfb4b317e68368...
6395,48e8faba4a168f803d7f16abb37ff877edd589b8081166...,db,6e02c6f25dc4d71560b03ba9077de4cb37383a1ec809de...
1596,3557add998c8762136da6e3a0af6489ed6425c3d91ca9e...,rpc,648b2a0c2f0a2c60f552c18abbc9ac747657751fc8db49...
5701,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,rpc,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...


In [14]:
test_df = remaining_df.drop(validation_df.index)  # drop the sampled rows to get the second DataFram
test_df

,um,rpctype,dm
51,95a6f7f8345e2eca31ee74ddc19d547e7fc0f5c8e65772...,rpc,3cab0a98767379fbcd059750c07ec42dbd0374cb114e0f...
57,e5155edb237fd9a04d9b322383113d21db13ade01944a5...,mc,676ca482c56de84046ce1d0076abd6cdb0f25778d036db...
67,95a6f7f8345e2eca31ee74ddc19d547e7fc0f5c8e65772...,mc,01d660afcfadafd587e20ec4c04ddbc7eb0de95643ba0e...
70,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,mc,01d660afcfadafd587e20ec4c04ddbc7eb0de95643ba0e...
77,63b8a25a4a9f80c7508c0db0d163af6d9e6c0746c9b7e2...,mc,6746853e39c5008988baf677bf27c846ee99173cafdec2...
...,...,...,...
16205,b39889af09ab89f4a7918a534348e405b52260bf198a43...,rpc,80df1ad95e41fc8fdef7671b6dd89e6b7ee5c982038a07...
16208,f2ff782d57371c090c294859e444ee4bdfb4b317e68368...,db,a2a5a9a0492f495f3dcc0bd933b0819cc8ac14289059d0...
16211,5b563df180af0e674902ffbeddf022580eee216f0a172f...,db,1abd717ffef1e3721f131eb3b37fc5c84fb9c52d0a42f3...
16213,9a0f0dd3e278f678cfa3f874a7c74af8206de653ec0396...,rpc,5ef671a7799d52d75c5c1b87d3542b5619c0ca7c9b3d80...


In [15]:
test_traces_df = complete_df.iloc[test_df.index].drop(columns=['um','rpctype','dm'],axis=1)
test_traces_df

,traceid
51,0b11355e15919238739898000e4315
57,0b11355e15919238739898000e4315
67,0b11355e15919238739898000e4315
70,0b11355e15919238739898000e4315
77,0b11355e15919238739898000e4315
...,...
16205,0b141b1f15919238823545000e868f
16208,0b1419ee15919237785631000e2cf2
16211,0b1418b215919236548593000e2658
16213,0b1418b215919236548593000e2658


In [16]:
test_traces = list(set(test_traces_df['traceid']))
len(test_traces)

1013

In [17]:
# complete_test_traces_df = complete_df[complete_df['traceid'].isin(test_traces)].drop(columns=['um','rpctype','dm'],axis=1)
# complete_test_traces_df

In [18]:
def create_train_test_trace(complete_df, trace):
    trace_df = complete_df[complete_df['traceid']==trace].drop_duplicates()
    trace_triplets_df = trace_df.drop(columns=['traceid'],axis=1).drop_duplicates()

    trace_entities = set(trace_triplets_df['dm']) | set(trace_triplets_df['um'])
    trace_relations = set(trace_triplets_df['rpctype'])
    trace_baseline = []
    for entity in trace_entities:
        trace_baseline.append(trace_triplets_df[trace_triplets_df['dm'] == entity].head(1))
        trace_baseline.append(trace_triplets_df[trace_triplets_df['um'] == entity].head(1))

    for relation in trace_relations:
        trace_baseline.append(trace_triplets_df[trace_triplets_df['rpctype'] == relation].head(1))


    trace_train_df = pd.concat(trace_baseline)
    trace_train_df = trace_train_df.drop_duplicates()

    trace_test_df = trace_triplets_df.drop(trace_train_df.index) 

    return trace_train_df, trace_test_df

In [19]:
# trace_train_df_list = []
# trace_test_df_list = [] 
# trace_id_list = []
# for trace in test_traces:
#     train_df, test_df = create_train_test_trace(complete_df, trace)
#     if len(test_df) > 0:
#         trace_train_df_list.append(train_df)
#         trace_test_df_list.append(test_df)
#         trace_id_list.append(trace)

In [20]:


@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [21]:
trace_train_df_list = []
trace_test_df_list = [] 
trace_id_list = []

def process_trace(trace):
    train_df, test_df = create_train_test_trace(complete_df, trace)
    if len(test_df) > 0:
        return (train_df, test_df, trace)
    else:
        return None

with tqdm_joblib(tqdm(desc="processing traces", total=len(test_traces))) as progress_bar:
    results = joblib.Parallel(n_jobs=-1)(
    joblib.delayed(process_trace)(trace) for trace in test_traces
)

for result in results:
    if result is not None:
        train_df, test_df, trace = result
        trace_train_df_list.append(train_df)
        trace_test_df_list.append(test_df)
        trace_id_list.append(trace)


processing traces: 100%|██████████| 1013/1013 [09:21<00:00,  1.81it/s]


In [22]:
def create_triplets(df):
    triplets = []
    for i in range(len(df)):
        head = df.iloc[i]['dm']
        tail = df.iloc[i]['um']
        rel = df.iloc[i]['rpctype']
        triplets.append([head,rel,tail])
    return triplets

In [23]:

jpype.startJVM()


def create_tsv(triplets, file_name):


    # Create Workbook object.
    workbook = Workbook(FileFormatType.TSV)

    # Access the first worksheet of the workbook.
    worksheet = workbook.getWorksheets().get(0)

    # Get the desired cell(s) of the worksheet and input the value into the cell(s).


    i = 1
    for elem in triplets: 
        worksheet.getCells().get("A"+str(i)).putValue(elem[0])
        worksheet.getCells().get("B"+str(i)).putValue(elem[1])
        worksheet.getCells().get("C"+str(i)).putValue(elem[2])
        i+=1


    # Save the workbook as TSV file.
    workbook.save(file_name)


    file = open(file_name,'r')  
    lines = file.readlines()  
    file.close()
    file = open(file_name,'w')  
    lines = lines[:-1]
    file.writelines(lines)
    file.close()

In [24]:

# Create a directory
directory = "MSCallGraph_traces"
if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"{directory} created successfully")
else:
    print(f"{directory} already exists")

    # Create a directory
directory = "MSCallGraph_traces/train"
if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"{directory} created successfully")
else:
    print(f"{directory} already exists")
    # Create a directory
directory = "MSCallGraph_traces/test"
if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"{directory} created successfully")
else:
    print(f"{directory} already exists")

MSCallGraph_0_traces created successfully
MSCallGraph_0_traces/train created successfully
MSCallGraph_0_traces/test created successfully


In [26]:
with tqdm(desc="writing train tsv", total=len(trace_id_list)):
    for elem in zip(trace_train_df_list,trace_id_list):
        create_tsv(create_triplets(elem[0]),f"MSCallGraph_traces/train/{elem[1]}_transductive_train.tsv")
        tqdm.update(1)

writing train tsv:   0%|          | 0/300 [00:07<?, ?it/s]


In [27]:
with tqdm(desc="writing test tsv", total=len(trace_id_list)):  
    for elem in zip(trace_test_df_list,trace_id_list):
        create_tsv(create_triplets(elem[0]),f"MSCallGraph_traces/test/{elem[1]}_transductive_test.tsv")
        tqdm.update(1)

writing test tsv:   0%|          | 0/300 [00:02<?, ?it/s]


In [28]:
jpype.shutdownJVM()

In [29]:

def zip_folder(folder_path, output_path):
    """
    Compresses a folder into a zip file.

    :param folder_path: The path to the folder to be compressed.
    :param output_path: The path and filename of the resulting zip file.
    """

    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        # Iterate over all the files in the folder and add them to the zip file.
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zip_file.write(os.path.join(root, file))

    print(f"{output_path} created successfully!")


# Example usage
folder_path = "MSCallGraph_traces"
output_path = folder_path+".zip"

print("Zipping")
zip_folder(folder_path, output_path)
print("Done")

Zipping
MSCallGraph_0_traces.zip created successfully!
Done
